![Pandas logo](img/pandas.svg)

# Reading and Writing Data

Pandas comes with I/O functions for a great many different formats.  You can get a feel for most of them with the Jupyter interactive help, by typing `pd.read_<tab>` to get a pop-up showing the functions with that prefix.  You can get a similar list of built-in output formats once you have loaded a DataFrame by typing `df.to_<tab>`.

A partial list of formats Pandas supports are:

- [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)
- [Excel](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html)
- [SQL](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html)
- [JSON](http://www.json.org/)
- [HDF5](https://www.hdfgroup.org/HDF5/)
- [pickle](https://docs.python.org/3/library/pickle.html)
- [msgpack](http://msgpack.org/)
- [Stata](https://en.wikipedia.org/wiki/Stata)
- [Apache Parquet](https://parquet.apache.org/documentation/latest/)
- [Google BigQuery](https://en.wikipedia.org/wiki/BigQuery)

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.training import *

## CSV

Delimited files—most commonly comma-separated—are a very common data format for shared data.  Pandas' `.read_csv()` function has a huge number of optional parameters to control how a data file is read into a DataFrame.  One obvious and important one is `sep=<delimiter>` which defaults to comma but could be any other character.

Let us look at a few options against a small file we saw earlier.  Run `pd.read_csv?` in a cell to get a lot of documentation of parameters.

In [ ]:
#pd.read_csv?

While CSV is widely used and even reasonably fast to read, it does have some limitations.  Values within CSV or similar text formats are not typed directly in the encoding.  We need either to rely on our tool, e.g. Pandas, to infer types for us; or we need to specify them explicitly on import.

In [ ]:
df = pd.read_csv('data/patient-records.csv')
df

In [ ]:
df.info()

In the `.info()` display, strings are generically Python objects.  For a person name, that is the best choice, but the date should be handled more specifically.

In [ ]:
# Parse the date in the "date colum"
df = pd.read_csv('data/patient-records.csv', parse_dates=['date'])
df.info()

In [ ]:
# Change column names, explicit types of some columns, choose index
patients = pd.read_csv('data/patient-records.csv', 
                       skiprows=1,
                       names=['Patient', 'Visit_Date', 'Weight', 'Height'],
                       parse_dates=['Visit_Date'],
                       dtype={'Height': np.float16, 'Weight': np.float16},
                       index_col='Visit_Date')
patients                        

In [ ]:
patients.info()

## Columns as Attributes

This simple DataFrame is an oportunity to note that if we happen to have column names that are valid Python identifiers, we can use a more compact attribute-style access rather than the dictionary-style indexing by column.  The square brackets are perfectly general, but the dot style sometimes looks nicer.

In [ ]:
patients[(patients.Height > 165) & (patients.Weight < 80)]

In [ ]:
patients[(patients['Height'] > 165) & (patients['Weight'] < 80)]

Once in a while you can get in trouble with the dot access because some attribute is both a column name and a DataFrame method or standard attribute.

In [ ]:
df = pd.DataFrame({"A": [1, 5, 7], 'std': [31, 52, 68]})
df.A

In [ ]:
df.std

In [ ]:
df.std()

In [ ]:
df['std']

## Write a DataFrame Back to CSV

After we have manipulated DataFrames in some manner, we can easily write them to new delimited files.

In [ ]:
import csv
patients.to_csv('tmp/patients.txt', sep='|', quoting=csv.QUOTE_NONNUMERIC)
!cat tmp/patients.txt

## SQL

There are a couple ways we can interact with SQL databases.  In particular, we can either read entire tables directly, or we can read the results of arbitrarily comple queries into DataFrames.  For the illustration here, an SQLite3 database is provided in the training materials.  However, the identical interfaces would work connecting to a distributed RDBMS (getting credentials correct for the connection object is a different topic, but once connection is established it is the same).

For reference, we first read in the Wisconsin breast cancer data set from CSV.

In [ ]:
cancer_csv = pd.read_csv('data/wisconsin.csv')
cancer_csv.info()

In the RDBMS we use, the data is broken out into separate tables for the features and the target.  In a more real world example, we would generally encounter many tables with a variety of foreign key constraints, triggers, indices, and so on.  Using some plain SQL (no Pandas), let us take a look at the schemata.

In [ ]:
from pprint import pprint
import sqlite3
db = sqlite3.connect('data/wisconsin.db')
cur = db.cursor()
cur.execute("SELECT * FROM sqlite_master;")
pprint(cur.fetchall())
del db

### Entire table

Unfortunately, Pandas often relies on the SQLAlchemy *object relational mapper* (ORM) rather than use the raw database connection.  Usually the wrapper does no harm, but ORMs *do impose* their world view too much in much of their API (not the parts used here, however).

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data/wisconsin.db')

In [ ]:
cancer_features_sql = pd.read_sql_table('features', engine, 
                                        index_col='Observation_No')
cancer_features_sql

### Based on a query

We can issue an arbitrary query rather than read an entire table into a DataFrame.  If memory permits, it is often friendlier to do filtering and other operations within Pandas rather than within SQL; but this style allows whichever combination is most useful to you.

Note that `pd.read_sql()` is polymorphic in taking either a table name or a query, then doing some basic pattern matching to figure out which kind of request it is.  We find it better practice to be explicit about the query involved though.

In [ ]:
sql = """
SELECT f.Observation_No, f.mean_radius, f.mean_texture, 
       f.mean_perimeter, t.benign 
FROM features f 
INNER JOIN target t 
WHERE f.Observation_No = t.Observation_No
LIMIT 100;
"""
cancer_with_target_sql = pd.read_sql_query(sql, engine, 
                                           index_col='Observation_No')
cancer_with_target_sql

## Write a DataFrame back to a table

After we have manipulated DataFrames in some manner, we can easily write them to tables in our RDBMS.

In [ ]:
!rm tmp/cancer-partial.sqlite
db = sqlite3.connect('tmp/cancer-partial.sqlite')
cancer_with_target_sql.to_sql('my_table', db)

pprint(db.execute("SELECT * FROM my_table LIMIT 5").fetchall())

## Excel Spreadsheets

Unfortunately, a great deal of data in the world lives in spreadsheets, and in Microsoft Excel specifically.  There are many things bad and broken about this format, but data comes as we get it.  For example:

* Very unreliable datatyping of columns and cells, especially if human entered data
* Poor handling of missing values
* Ragged, sparse, and irregular use of rectangular grid of rows/columns
* Computation expressed in distributed and hard-to-trace manner
* Inconsistent uses of tabs for hierarchy or structure
* Limitation on number of rows and inflexible datatypes
* Mediocre heuristics for inference of datatypes
* Use of columns for values of inconsistent datatypes
* **Extremely** slow compared to nearly every data format

Pandas allows us to read Excel sheets, and even to clean up the bad data we find once they are loaded into DataFrames.

In [ ]:
cancer_excel = pd.read_excel('data/wisconsin.xlsx', 
                             sheet_name=['Features', 'Target'],
                             index_col='Observation_No')
cancer_excel['Features'].head()

In [ ]:
cancer_excel['Target'].head()

## External Data

Many of the `.read_<format>()` methods in Pandas are happy to accept URLs following a number of schemata (HTTP, HTTPS, FTP, S3, file, and others).

In [ ]:
URL = ("https://bitbucket.org/davidmertz/sample-data/"
       "raw/9fb79b1e993e58ba7e3137f63726bb4a19bc0e8e/patient-records.csv")
pd.read_csv(URL)

## JSON

Pandas DataFrames are inherently two dimensional grids of values.  This is very broadly useful, and maps well to formats like CSV, spreadsheets, and relational database tables or queries.  However, other data is hierarchical or has other structures.  

One of the most widely used hierarchical formats is Javascript Object Notation (JSON).  Flat grids are a subset, conceptually, of all possible nested structures.  But JSON is extermely widely used for data transmitted between web services, and for other purposes, so Pandas provides a number of ways to read/write JSON data that is "conceptually flat."

Some JSON formats result in lossy round-tripping with DataFrames since they do not represent all the components of a DataFrame.  Another thing to trip over is that JSON types by values; so e.g. if all values in a column are *equal to an integer* the writer will cast as integer.

In [ ]:
df = pd.DataFrame({'col1': [1.0, 2, 3], 'col2': [5.1, 6.2, 7.3]},
                  index=['A', 'B', 'C'])
df.index.name = "Letter"
df

In [ ]:
style = 'split'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

In [ ]:
style = 'records'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

In [ ]:
style = 'index'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

In [ ]:
style = 'columns'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

In [ ]:
style = 'split'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

In [ ]:
style = 'table'
json = df.to_json(orient=style)
pprint(json)
pd.read_json(json, orient=style)

## Parquet and Arrow

Pandas can also read both the Parquet and Arrow formats, which are **far** higher performance than formats like Excel, CSV, JSON, and so on.  Both of these formats are column oriented and strictly data typed, which make them well suited to working on their data in data frames.

These two formats are slightly different creatures, however.  Arrow is a language-independent *memory format* so that varying libraries, across varying programming languages, can access the same data without performing copying.  For example, both Python Pandas and R dplyer can operate on the same memory, in principle.  Arrow also defines a serialization format, called Feather, which is essentially just a direct map of bytes in memory to bytes on disk.

Parquet is column-oriented *data file format* designed with very efficient data compression and encoding schemes.  However, when read into memory, Parquet data simply has the native in-memory layout of your data frame library. For modern versions of Pandas, that memory layout simply *is* Arrow.

In most cases, you should prefer to read and write Parquet rather than Arrow.  Its compression causes its files to use less disk space, and usually to read and write more quickly as well since CPU compression speeds greatly outpace disk write speeds.  Most of the same large range of tools, across many programming languages, support both Parquet and Arrow.

In [ ]:
cancer_csv.to_parquet("tmp/wisconsin.parquet")
pd.read_parquet("tmp/wisconsin.parquet")

In [ ]:
cancer_csv.to_feather("tmp/wisconsin.feather")
pd.read_feather("tmp/wisconsin.feather")

# Exercises

For a first few exercises, you will read and write from a variety of formats to get a feel for them.  Moreover, the speed of different formats can be dramatically different, so time operations against different formats (both reading and writing) using the `%timeit` magic in Jupyter.

Note that the `%timeit` macro creates its own namespace for operations, so when you want the actual result from an operation to work with more, you will need to run the function or method outside of the macro also.

## Obtain Data

Download the datasets provided as CSV URLs, and also save them locally as CSV.  Using the `tmp/` relative path within this training repository is a good choice of location to save files at.

* Is loading from local storage faster than from remote URL?
* What do you conclude about the main speed limitations in loading data from CSV?
* Does the same pattern hold for the smaller "Baby Names" dataset and the larger "Daily Temperatures" dataset?
* Were the inferred datatypes all the best choices for the respective fields?
* If you wanted to read some fields as different datatypes, how would you do that?
* What effect does file compression have on reading and writing CSV format?

In [ ]:
# This dataset obtained from https://www.data.gov/
url = ("https://bitbucket.org/davidmertz/sample-data/raw/"
       "1bd7d3fbfc6842eb067ea9a9354b6b4e5b8597ab/Popular_Baby_Names.csv")
babynames = pd.read_csv(url)
babynames.to_csv('tmp/babynames.csv')
%timeit pd.read_csv(url)

The United States [National Oceanic and Atmospheric Administration (NOAA)](https://www.noaa.gov/) provides a number of useful datasets.  We will work with a subset of daily temperatures worldwide during 2019, which was obtained from https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/.  Notice that Pandas will seamlessly read compressed CSV files as well.

In [ ]:
url = ("https://bitbucket.org/davidmertz/sample-data/raw/"
       "61872271984f66e3094c367cf90dfc4875a22e8d/NOAA-2019-partial.csv.gz")
temperatures = pd.read_csv(url)
%timeit pd.read_csv(url)

## Play with Different Formats

Save these datasets in other formats we have discussed, or even only mentioned in passing.  

* How does the speed of reading and writing compare?
* Do you lose any information round-tripping between various formats? (datatypes are information too)
* How do you expect performance to scale if you were to move from half-a-million rows to a hundred million rows?
* Would the answer vary across different formats?

In [ ]:
# Experiment here...

## Explain Advantages and Disadvantages

Explain to the person sitting next to you (or write in a cell below) what advantages and disadvantages you find in using different storage formats for Pandas DataFrames.  It is fair if one of the advantages is "my boss/work requires me to use X."

*Explanation of virtues and demerits of formats ...*

## Working with Data

Using ideas we have presented in passing and patterns you learned working with NumPy, try to extract only the temperatures for January, February, and March.

In [ ]:
# Yes, the solstice and equinox do not line up with months exactly
# If you would like to find the accurate cut-offs for "winter", please do
winter = ... # something using `temperatures`

Being data from the United States, temperatures are unfortunately provided in Fahrenheit. At least it would appear so:

In [ ]:
temperatures['TEMP'].max(), temperatures['TEMP'].min()

As mentioned, the data is a subset of the full year data. In later lessons we try to characterize the distributions of dates, longitudes, lattitudes, elevaations, and so on.  But, we do know:

> The official highest recorded temperature is now 56.7℃ (134℉), which was measured on 10 July 1913 at Greenland Ranch, Death Valley, California, USA.

So 92.2 would be quite low if the whole earth was represented as ℉.  But the number is vastly higher than any earth-surface ℃.

For extra credit, see if you can figure out how to add a new column to the `temperatures` DataFrame called `TEMP(C)` with the obvious meaning and correct values.

In [ ]:
# Add a column for Celcius temperatures
...

Write your improved data back to disk in your preferred data format.